In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import pickle

'''
각 레이어에서 '훈련 피처 맵'과 '정분류 테스트 피처 맵'과 '오분류 테스트 피처 맵'은 모두 같은 형태이어야 한다.
예를 들어, 첫번째 레이어에서 훈련 피처 맵(100, 12, 12) 정분류 테스트 피처 맵(30, 12, 12), 오분류 테스트 피처 맵(20, 12, 12)
와 같이 인덱스 첫번째인 개수는 다르더라도 이차 넘파이 배열은 같아야 한다.
'''

class FMD():
    # directory
    # root_dir: 관련된 데이터가 모두 저장되어 있는 디렉토리 경로
    # origin_dir: train, rtest, wtest가 있는 디렉토리
    # eval_dir: 거리를 잴 데이터가 있는 디렉토리
    root_dir=""; origin_dir=""; eval_dir=""

    # data_infos
    # 데이터 타입에 대한 이름
    origin_names = ["train", "rtest", "wtest"]
    eval_names = []
    # (이하) 대괄호로 쳐져있는 배열은 나중에 넘파이 배열이 될 수 있다.
    # (origin_, test_)K: 피처 맵의 개수, L: 레이어의 개수(0~L-1는 각 레이어의 인덱스)
    # shape: 레이어의 넘파이 모양
    # 아래 (origin_, test_)K, L, shape을 'data_infos'라고 하자.
    origin_K={"train": 0, "rtest": 0, "wtest": 0}; eval_K={}; L=0; shape=[]
    # (이하) 훈련 데이터, 정분류 테스트 데이터, 오분류 테스트 데이터에서 데이터라는 용어를 생략하고 적었다.
    # 아래의 배열들은 모두 파이썬 배열이고 첫 번째 인자에 layer가 들어간다.
    # TFM_mean: 훈련 평균 피처 맵(베이스 피처 맵)
    # RFM_mean: 정분류 테스트 평균 피처 맵
    # WFM_mean: 오분류 테스트 평균 피처 맵

    # FM_means
    TFM_mean=[]; RFM_mean=[]; WFM_mean=[]
    # TAM: 훈련 활성화 맵       , RAM: 정분류 테스트 활성화 맵
    # WAM: 오분류 테스트 활성화 맵

    # AMs와 그와 관련된 것
    TAM=[]; RAM=[]; WAM=[]
    # alpha는 거리 계산을 위한 인덱스를 고르기 위해 필요한 변수이다.
    # w_minus_r_max(= max(w-r))는 나중에 alpha 값에 의존하여 최대로 만들어진다.
    # alpha_min, alpha_max는 각 레이어에서의 alpha가 가질 수 있는 최소값 최대값을 나타낸 것이다.
    # [hyper parameter] alpha_slice은 alpha_min에서 alpha_max로 몇 번의의 간격으로 도착할지 알려주는 변수임.
    alpha=[]; alpha_slice=[]; w_minus_r_max=[]
    alpha_min=[]; alpha_max=[]
    # DAM_indexes는 나중에 거리 계산할 때 쓰이는 2차원 인덱스들의 집합이다.
    # 각 원소는 피처 맵의 한 원소의 인덱스를 나타낸다.
    # DAM: 거리 활성화 맵, DAM는 거리 계산을 위한 인덱스만 활성된 맵이다.
    # 각 레이어마다 튜플들 세트가 있어 함. np.array의 item 메소드를 사용할 것이기 때문
    # [hyper parameter] DAM_select는 DAM를 고르는 방법을 알려줌.
    DAM_indexes=[]; DAM=[]; DAM_select=[]

    # index_infos
    # '오분류 테스트' 각 인덱스에서의 거리에 대한 중간값, 평균, 표준편차, 최소값, 최대값에 대한 배열은
    # 각 mid, mean, std, min, max임
    # 인덱스마다 달라진는 값이니 총칭해서 'index_infos'라고 하자.
    mid=[]; mean=[]; std=[]; min=[]; max=[]
    
    # layer_infos
    # W: 레이어 피처맵 거리 가중치, ld_kind: 레이어 피처맵 거리 계산 방법
    # [hyper parameter] W는 각 레이어의 피처 맵에 곱할 weight 중요도이다
    # [hyper parameter] ld_kind는 각 레이어에 대한 피처 맵을 구하는 방법을 저장한다.
    W=[]; ld_kind=[]

    # FMDC
    # FMDC 피처 맵 거 기준으로 어떤 데이터가 나중에 오분류 될 거 같은지 판단함.
    # FMDC는 거리이므로 음수가 될 수 없음, 따라서, FMDC가 -1이라는 것은 아직 계산하지 않았다는 의미임.
    FMDC=-1

    def __init__(self, root_dir_=""):
        # root 디렉토리 입력이 없다면 return
        if root_dir_=="":
            print("루트 경로를 설정해주세요.")
            return
        # root 디렉토리
        self.root_dir = root_dir_

        # origin 디렉토리
        self.origin_dir = f"{self.root_dir}/origin"
        # 훈련을 저장하는 디렉토리
        self.train_dir=f"{self.origin_dir}/{self.origin_names[0]}"
        # 정분류 테스트를 저장하는 디렉토리
        self.rtest_dir=f"{self.origin_dir}/{self.origin_names[1]}"
        # 오분류 테스트를 저장하는 디렉토리
        self.wtest_dir=f"{self.origin_dir}/{self.origin_names[2]}"
        
        # eval 디렉토리
        self.eval_dir = f"{self.root_dir}/eval"

        # 객체를 불러오거나 저장할 디렉토리 생성
        os.system("mkdir ./models")

    def fit(self):
        # 객체의 속성 초기화
        # root_dir의 데이터가 올바르다면 init이 에러 없이 완료됨
        self.set_data_infos()
        self.set_FM_means()
        self.set_AMs()
        self.set_index_infos()
        self.set_fmdc()

    def set_root_dir(self, root_dir_):
        self.root_dir = root_dir_

        # origin 디렉토리
        self.origin_dir = f"{self.root_dir}/origin"
        # 훈련을 저장하는 디렉토리
        self.train_dir=f"{self.origin_dir}/{self.origin_names[0]}"
        # 정분류 테스트를 저장하는 디렉토리
        self.rtest_dir=f"{self.origin_dir}/{self.origin_names[1]}"
        # 오분류 테스트를 저장하는 디렉토리
        self.wtest_dir=f"{self.origin_dir}/{self.origin_names[2]}"
        
        # eval 디렉토리
        self.eval_dir = f"{self.root_dir}/eval"

    def create_practice(self):
        '''
        아마도 나중에 adapter를 만들 때 사용될 수 있을 것 같음.
        '''

        # 연습 데이터 랜덤 시드 설정
        random_seed_id = 42
        np.random.seed(random_seed_id)
        # 연습 데이터 범위 설정
        random_start = 1; random_end = 1000000

        # 디렉토리 설정
        root_dir = "practice"
        origin_dir = f"{root_dir}/origin"
        eval_dir = f"{root_dir}/eval"
        # 디렉토리 생성
        os.system(f"mkdir {root_dir}")
        os.system(f"mkdir {origin_dir}")
        os.system(f"mkdir {eval_dir}")

        # origin 데이터 종류 적기
        origin_names="train rtest wtest"
        # origin_K 각 데이터 종류마다의 개수
        origin_K="10000 2000 4000"
        # eval 데이터 종류 적기
        eval_names="rotation_90 brightness_10 whiteness_10"
        # eval_K 각 데이터 종류마다의 개수
        eval_K="12000 4000 6000"
        # L: 레이어의 개수, shape: 레이어의 모양
        L="3";
        shape = "12 12" + "\n"
        shape += "12 2 3 4" + "\n"
        shape += "24 8"

        # data_infos.txt 파일로 저장
        # origin 데이터 종류는 train rtest wtest로 이미 정해져 있으므로 파일에 저장하지 않음
        os.system(f"touch {root_dir}/data_infos.txt")
        data_infos_txt = open(f"{root_dir}/data_infos.txt", 'w')
        data_infos_txt.write(origin_K + "\n")
        data_infos_txt.write(eval_names + "\n")
        data_infos_txt.write(eval_K + "\n")
        data_infos_txt.write(L + "\n")
        data_infos_txt.write(shape)
        data_infos_txt.close()

        # 자료구조를 바꾸어 for문 활용을 쉽게 하기
        origin_names = origin_names.split()
        origin_K = list(map(int,origin_K.split()))
        origin_K_list = origin_K
        origin_K = {}
        for i in range(len(origin_names)):
            origin_K[origin_names[i]] = origin_K_list[i]
        
        eval_names = eval_names.split()
        eval_K = list(map(int,eval_K.split()))
        eval_K_list = eval_K
        eval_K = {}
        for i in range(len(eval_names)):
            eval_K[eval_names[i]] = eval_K_list[i]

        L=int(L)

        shape = shape.split('\n')
        for ith in range(len(shape)):
            shape[ith] = list(map(int,shape[ith].split()))

        # origin 연습 데이터 생성
        for origin_name in origin_names:
            os.system(f"mkdir {origin_dir}/{origin_name}")
            for k in range(origin_K[origin_name]):
                for l in range(L):
                    np.save(f"{origin_dir}/{origin_name}/{origin_name}_{k}_{l}.npy",
                            np.random.randint(random_start, random_end, size=shape[l]) - np.random.random(shape[l]))
        
        # eval 연습 데이터 생성
        for eval_name in eval_names:
            os.system(f"mkdir {eval_dir}/{eval_name}")
            for k in range(eval_K[eval_name]):
                for l in range(L):
                    np.save(f"{eval_dir}/{eval_name}/{eval_name}_{k}_{l}.npy",
                            np.random.randint(random_start, random_end, size=shape[l]) - np.random.random(shape[l]))

    def set_data_infos(self):
        # root dir의 data_infos.txt 열기
        data_infos = open(f"{self.root_dir}/data_infos.txt", 'r')
        data_infos_str = data_infos.read()
        data_infos_list = data_infos_str.split('\n')
        # 0th: origin_K 
        origin_K = list(map(int, data_infos_list[0].split()))
        origin_name_K_zip = zip(self.origin_names, origin_K) 
        for origin_name, origin_K in origin_name_K_zip:
            self.origin_K[origin_name] = origin_K
        # 1th: eval_names
        self.eval_names = data_infos_list[1].split()
        # 2th: eval_K
        eval_K = list(map(int, data_infos_list[2].split()))
        eval_name_K_zip = zip(self.eval_names, eval_K) 
        for eval_name, eval_K in eval_name_K_zip:
            self.eval_K[eval_name] = eval_K
        # 3th: L
        self.L = int(data_infos_list[3])
        # 4+0th ~ 4+(L-1)th: shape
        for l in range(self.L):
            shape_l = list(map(int,data_infos_list[4+l].split()))
            self.shape.append(shape_l)
        # root dir의 data_infos_str.txt 닫기
        data_infos.close()
        
        # 레이어 피처 맵을 구하는 방식을 mdld로 모두 통일
        for l in range(self.L):
            self.ld_kind.append("mdld")
        # W를 균등하게 만듦
        for l in range(self.L):
            self.W.append(1/self.L)

    def set_FM_means(self):
        '''
        self.TFM_mean, self.RFM_mean, self.WFM_mean를 포인터와 같이 추적할 수 있다면
        일반화하여 가독성을 위해 코드를 3배 더 줄일 수 있을 것 같은데 어떻게 하는지 모르겠다.
        '''
        train = self.origin_names[0]; rtest = self.origin_names[1]; wtest = self.origin_names[2]
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            TFM_mean_l = np.zeros(self.shape[l])
            self.TFM_mean.append(TFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 TFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L):
            TFM_0_l = np.load(f"{self.train_dir}/{train}_{0}_{l}.npy")
            self.TFM_mean[l] = self.TFM_mean[l] + TFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.origin_K[train]):
            for l in range(self.L):
                TFM_k_l = np.load(f"{self.train_dir}/{train}_{k}_{l}.npy")
                self.TFM_mean[l] = (self.TFM_mean[l]*k + TFM_k_l)/(k+1)
        
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            RFM_mean_l = np.zeros(self.shape[l])
            self.RFM_mean.append(RFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 RFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L):
            RFM_0_l = np.load(f"{self.rtest_dir}/{rtest}_{0}_{l}.npy")
            self.RFM_mean[l] = self.RFM_mean[l] + RFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.origin_K[rtest]):
            for l in range(self.L):
                RFM_k_l = np.load(f"{self.rtest_dir}/{rtest}_{k}_{l}.npy")
                self.RFM_mean[l] = (self.RFM_mean[l]*k + RFM_k_l)/(k+1)
        
        # 각 레이어의 피처 맵을 0으로 초기화하여 생성
        for l in range(self.L):
            WFM_mean_l = np.zeros(self.shape[l])
            self.WFM_mean.append(WFM_mean_l)
        # 파일에 저장된 훈련 데이터 피처 맵 가지고 와서 WFM_mean 만들기
        # 0번 째 데이터를 TFM_mean에 넣은 후
        for l in range(self.L):
            WFM_0_l = np.load(f"{self.wtest_dir}/{wtest}_{0}_{l}.npy")
            self.WFM_mean[l] = self.WFM_mean[l] + WFM_0_l
        # 1~K번 째 데이터로 TFM_mean을 구한다.
        for k in range(1, self.origin_K[wtest]):
            for l in range(self.L):
                WFM_k_l = np.load(f"{self.wtest_dir}/{wtest}_{k}_{l}.npy")
                self.WFM_mean[l] = (self.WFM_mean[l]*k + WFM_k_l)/(k+1)

    def set_AMs(self):
        # alpha, w_minus_r_max를 0으로 초기화
        self.alpha = np.zeros(self.L)
        self.w_minus_r_max = np.zeros(self.L)
        # [hyperparameter] alpha slice를 일단 1000으로 함
        for l in range(self.L):
            self.alpha_slice.append(1000)
        # alpha의 최소값과 최대값을 구함
        self.alpha_min = np.zeros(self.L)
        self.alpha_max = np.zeros(self.L)
        for l in range(self.L):
            self.alpha_min[l] = np.array([self.RFM_mean[l].min(),
                                         self.TFM_mean[l].min(),
                                         self.WFM_mean[l].min()]).min()
            self.alpha_max[l] = np.array([self.RFM_mean[l].max(),
                                         self.TFM_mean[l].max(),
                                         self.WFM_mean[l].max()]).max()

        # TAM, RAM, WAM을 0으로 초기화 함
        for l in range(self.L):
            TAM_l = np.zeros(self.shape[l])
            self.TAM.append(TAM_l)
        for l in range(self.L):
            RAM_l = np.zeros(self.shape[l])
            self.RAM.append(RAM_l)
        for l in range(self.L):
            WAM_l = np.zeros(self.shape[l])
            self.WAM.append(WAM_l)

        # w-r가 최대가 되는 alpha, TAM, RAM, WAM을 찾음
        for l in range(self.L):
            # range 부분 고칠 필요가 있음
            a_min_l = self.alpha_min[l]; a_max_l = self.alpha_max[l]
            a_slice_l = self.alpha_slice[l]
            interval_l = (a_max_l - a_min_l)/a_slice_l
            # range(a_slice_l+1) 해야 a_min_l 부터 a_max_l 까지 감
            for alpha_l in [a_min_l + interval_l*s for s in range(a_slice_l+1)]:
                # 각 AM에서 alpha보다 큰 인덱스 부분은 1, 작거나 같은 인덱스 부분은 0로 됨
                TAM_l = np.array(self.TFM_mean[l] > alpha_l, dtype="int32")
                RAM_l = np.array(self.RFM_mean[l] > alpha_l, dtype="int32")
                WAM_l = np.array(self.WFM_mean[l] > alpha_l, dtype="int32")

                TAM_l_minus_RAM_l = TAM_l - RAM_l
                TAM_l_minus_WAM_l = TAM_l - WAM_l

                r_l = len(np.where(TAM_l_minus_RAM_l == 1)[0])
                w_l = len(np.where(TAM_l_minus_WAM_l == 1)[0])
                
                if w_l - r_l > self.w_minus_r_max[l]:
                    # w-r가 이전의 w-r보다 클 때
                    # alpha, w-r, TAM, RAM, WAM를 최신화함.
                    self.w_minus_r_max[l] = w_l - r_l
                    self.alpha[l] = alpha_l
                    self.TAM[l] = TAM_l
                    self.RAM[l] = RAM_l
                    self.WAM[l] = WAM_l

        # TAM, RAM, WAM를 이용하여 DAM를 구함
        # DAM에 WAM를 deep copy 복사함
        for l in range(self.L):
            DAM_l = self.WAM[l].copy()
            self.DAM.append(DAM_l)
        # [hyperparameter] 일단 모든 레이어를 'and' 방식으로 함
        for l in range(self.L):
            self.DAM_select.append("and")
        # 선택하려는 방식('and', 'or', 등등)대로 DAM를 고름
        for l in range(self.L):
            if self.DAM_select[l] == "and":
                TAM_l_and_RAM_l = np.logical_and(self.TAM[l], self.RAM[l])
                np.place(self.DAM[l], TAM_l_and_RAM_l, 0)
            elif self.DAM_select[l] == "or":
                TAM_l_or_RAM_l = np.logical_or(self.TAM[l], self.RAM[l])
                np.place(self.DAM[l], TAM_l_or_RAM_l, 0)

        # DAM_indexes를 지정함
        for l in range(self.L):
            nonzero_DAM_l = np.nonzero(self.DAM[l])
            DAM_indexes_l = np.empty((1,len(nonzero_DAM_l[0])), dtype="int32")

            for i in range(len(nonzero_DAM_l)):
                DAM_indexes_l = np.append(DAM_indexes_l, nonzero_DAM_l[i].reshape(1,-1), axis=0)
            
            DAM_indexes_l = list(DAM_indexes_l[1:].T) # 처음 배열은 쓰레기 값이라 버림

            for i in range(len(DAM_indexes_l)):
                DAM_indexes_l[i] = tuple(DAM_indexes_l[i])

            self.DAM_indexes.append(DAM_indexes_l)
    
    def set_index_infos(self):
        wtest = self.origin_names[2]

        for l in range(self.L):
            # mid_l, mean_l, std_l, min_l, max_l를 모두 -1로 초기화
            # 거리가 -1(음수)이 될 수 없으므로 -1인 부분은 초기화 되지 않은 인덱스 값임
            mid_l = np.zeros(self.shape[l]); mid_l = mid_l - 1
            mean_l = np.zeros(self.shape[l]); mean_l = mean_l - 1
            std_l = np.zeros(self.shape[l]); std_l = std_l - 1
            min_l = np.zeros(self.shape[l]); min_l = min_l - 1
            max_l = np.zeros(self.shape[l]); max_l = max_l - 1
            for DAM_index_l in self.DAM_indexes[l]:
                ED_array_l_DAM_index = np.empty((0), dtype="float")
                for k in range(self.origin_K[wtest]):
                    WFM_k_l = np.load(f"{self.wtest_dir}/{wtest}_{k}_{l}.npy")
                    # append를 많이 사용하면 속도가 줄어듦
                    ED_array_l_DAM_index = np.append(ED_array_l_DAM_index, abs(self.TFM_mean[l].item(DAM_index_l)
                                                                                 - WFM_k_l.item(DAM_index_l))) # TFM_mean = BFM

                mid_l.itemset(DAM_index_l, sorted(ED_array_l_DAM_index)[self.origin_K[wtest] // 2])
                mean_l.itemset(DAM_index_l, ED_array_l_DAM_index.mean())
                std_l.itemset(DAM_index_l, ED_array_l_DAM_index.std())
                min_l.itemset(DAM_index_l, ED_array_l_DAM_index.min())
                max_l.itemset(DAM_index_l, ED_array_l_DAM_index.max())
            
            self.mid.append(mid_l)
            self.mean.append(mean_l)
            self.std.append(std_l)
            self.min.append(min_l)
            self.max.append(max_l)

    def mdld(self, dir1, dir2, k, l):
        '''
        중앙값으로 나눠서 구한 레이어 거리(middle division layer distance)

        dir1은 root 디렉토리 기준으로 깊이가 1인 것
        dir2은 root 디렉토리 기준으로 깊이가 2인 것
        k는 몇 번째 데이터인지 알려줌, l은 몇번째 레이어인지 알려줌
        예시: MDD("origin", "train", 1, 2)은
        root 디렉토리/origin/train/train_1_2에 대한 레이어 거리임.
        '''
        mdld = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')

        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mid_index = self.mid[l].item(index)

            mdld += (length_index / mid_index)**2
        
        return mdld
    
    def normld(self, dir1, dir2, k, l):
        '''
        정규화를 시켜서 구한 레이어 거리(normal layer distance)
        '''
        normld = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')
        
        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mean_index = self.mean[l].item(index)
            std_index = self.std[l].item(index)

            normld += math.exp((length_index - mean_index) / std_index)
        
        return normld

    def mmsld(self, dir1, dir2, k, l):
        '''
        min-max으로 표준화한 후 shift하여 구한 레이어 거리(min max shift layer distance)
        '''
        mmsld = 0
        dir2_FM_k_l = np.load(f'{self.root_dir}/{dir1}/{dir2}/{dir2}_{k}_{l}.npy')
        
        for index in self.DAM_indexes[l]:
            length_index = abs(self.TFM_mean[l].item(index) - dir2_FM_k_l.item(index))
            mid_index = abs(self.TFM_mean[l].item(index) - self.mid[l].item(index))
            min_index = self.min[l].item(index)
            max_index = self.max[l].item(index)

            min_max_index = (length_index - min_index) / (max_index - min_index)
            min_max_mid_index = (mid_index - min_index) / (max_index - min_index)
            
            if length_index <= mid_index:
                mmsld += (min_max_index + 0)**2
            else:
                mmsld += (min_max_index + 1 - min_max_mid_index)**2
        
        return mmsld
    
    def fmd(self, dir1, dir2, k):
        fmd=0
        lds=[]
        # 각 레이어에 대한 레이어 피처 맵 거리 계산법으로 레이어 피처 맵 계산
        for l in range(self.L):
            ld_l = 0
            if self.ld_kind[l] == 'mdld':
                ld_l = self.mdld(dir1, dir2, k, l)
            elif self.ld_kind[l] == 'normld':
                ld_l = self.normld(dir1, dir2, k, l)
            elif self.ld_kind[l] == 'mmsld':
                ld_l = self.mmsld(dir1, dir2, k, l)
            lds.append(ld_l)
        # 레이어 피처 맵마다 weight를 줌
        for l in range(self.L):
            fmd += self.W[l]*lds[l]

        return fmd
    
    def set_fmdc(self):
        fmds=[]
        for k in range(self.origin_K["wtest"]):
            fmds.append(self.fmd('origin', 'wtest', k))
        fmds = np.array(fmds)
        
        self.FMDC = fmds.min()

    def show_all(self):
        self.show_data_infos()
        self.show_FM_means()
        self.show_AMs_and_related()
        self.show_hyper_parameter()
        self.show_index_infos()
        self.show_layer_infos()
        self.show_FMDC()
        self.show_dirs()
        
    def show_data_infos(self):
        print("self.origin_names"); print(self.origin_names)
        print("self.origin_K"); print(self.origin_K)
        print("self.eval_names"); print(self.eval_names)
        print("self.eval_K"); print(self.eval_K)
        print("self.L"); print(self.L)
        print("self.shape"); print(self.shape)

    def show_FM_means(self):
        print("self.TAM_mean"); print(self.TFM_mean)
        print("self.RAM_mean"); print(self.RFM_mean)
        print("self.WAM_mean"); print(self.WFM_mean)

    def show_AMs_and_related(self):
        print("self.TAM"); print(self.TAM)
        print("self.RAM"); print(self.RAM)
        print("self.WAM"); print(self.WAM)

        print("self.alpha_slice"); print(self.alpha_slice)
        print("self.alpha_min"); print(self.alpha_min)
        print("self.alpha"); print(self.alpha)
        print("self.alpha_max"); print(self.alpha_max)
        print("self.w_minus_r_max"); print(self.w_minus_r_max)

        print("self.DAM_select"); print(self.DAM_select)
        print("self.DAM"); print(self.DAM)
        print("self.DAM_indexes"); print(self.DAM_indexes)

    def show_hyper_parameter(self):
        print("self.alpha_slice"); print(self.alpha_slice)
        print("self.DAM_select"); print(self.DAM_select)
        print("self.ld_kind"); print(self.ld_kind)
        print("self.W"); print(self.W)

    def show_index_infos(self):
        print("self.mid"); print(self.mid)
        print("self.mean"); print(self.mean)
        print("self.std"); print(self.std)
        print("self.min"); print(self.min)
        print("self.max"); print(self.max)

    def show_layer_infos(self):
        print("self.ld_kind"); print(self.ld_kind)
        print("self.W"); print(self.W)

    def show_FMDC(self):
        print("self.FMDC"); print(self.FMDC)

    def show_dirs(self):
        print("self.root_dir"); print(self.root_dir)
        
        print("self.origin_dir"); print(self.origin_dir)
        print("self.train_dir"); print(self.train_dir)
        print("self.rtest_dir"); print(self.rtest_dir)
        print("self.wtest_dir"); print(self.wtest_dir)

        print("self.eval_dir"); print(self.eval_dir)

    # 객체를 저장하고 불러오는 것은 검증을 해야함
    def save_object(self, model_name):
        # 인자로 지정된 경로와 이름으로 파일 저장
        with open(f"./models/{model_name}.pickle", "wb") as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
    def load_object(self, model_name):
        # 인자로 지정된 경로와 이름으로 파일 불러오기
        with open(f"./models/{model_name}.pickle", "rb" ) as f:
            object_ = pickle.load(f)
            self = object_

In [55]:
fmd1 = FMD("practice")
# fmd1.create_practice()
fmd1.fit()
fmd1.show_all()

self.origin_names
['train', 'rtest', 'wtest']
self.origin_K
{'train': 10000, 'rtest': 2000, 'wtest': 4000}
self.eval_names
['rotation_90', 'brightness_10', 'whiteness_10']
self.eval_K
{'rotation_90': 12000, 'brightness_10': 4000, 'whiteness_10': 6000}
self.L
3
self.shape
[[12, 12], [12, 2, 3, 4], [24, 8]]
self.TAM_mean
[array([[498403.96333842, 498059.2321978 , 499645.77094002,
        503823.54264013, 499501.26100772, 490857.89327967,
        496314.59603951, 498583.11816414, 499326.45549089,
        506111.67328193, 498456.49989894, 497752.29064165],
       [500306.78316394, 498568.99691985, 500183.62100267,
        503938.35867494, 496130.15966179, 496656.31973091,
        504815.62538443, 503234.95427865, 502036.59319452,
        502635.46279352, 502812.77126256, 501269.97150107],
       [497626.2797457 , 500325.86552544, 499221.41249677,
        504947.65104853, 501424.62407036, 499397.50505772,
        496724.09044353, 503286.51379893, 500753.26560834,
        501153.7972885 , 50

In [45]:
fmds=[]
for k in range(fmd1.origin_K["wtest"]):
    fmds.append(fmd1.fmd('origin', 'wtest', k))
fmds = np.array(fmds)
print(sorted(fmds))

[59.95204687048876, 61.167696756847114, 61.52537189720489, 61.69467909844592, 62.17282349153338, 62.40702631740806, 62.59652273200281, 63.01703753166527, 63.36274908152961, 63.41185170318934, 63.519120046339204, 63.57688351870878, 63.65161987528373, 63.653579793866705, 63.660591434916014, 63.87056098790187, 63.87311244247275, 63.99244286667604, 64.03737673791295, 64.20517388769692, 64.2481528007278, 64.27750780553733, 64.32302186139792, 64.49161777594105, 64.49543022530636, 64.56885054887556, 64.60271189193861, 64.62917579714481, 64.67212062505236, 64.69484953691762, 64.72930988522714, 64.75134836624574, 64.97311509560029, 65.02323901924015, 65.02445982769166, 65.11076089393936, 65.11279525565965, 65.13978574040496, 65.2669254428464, 65.28014742203794, 65.33347933767223, 65.36054193388233, 65.39057996072923, 65.55319823981691, 65.56485738293749, 65.59083140938245, 65.62602440252759, 65.65759109911515, 65.84939084188815, 65.86346078016332, 65.86637176880795, 65.87990126813165, 65.901373

In [43]:
fmd1.FMDC

59.95204687048876

In [33]:
for l in range(fmd1.L):
    print((0.23+0.1*l)*fmd1.mdld('origin', 'train', 1, l))
print()
for l in range(fmd1.L):
    print((0.23+0.1*l)*fmd1.normld('origin', 'train', 2, l))
for l in range(fmd1.L):
    print((0.23+0.1*l)*fmd1.mmsld('origin', 'train', 3, l))

16.576895057101275
24.76034177985256
31.32217085626794

16.57337573902506
29.831695847176086
31.571327921614866

12.41551502539908
17.436925081794687
19.084123920142577


In [34]:
fmd1.save_object("fmd1")

In [35]:
fmd2 = FMD("practice")
fmd2.load_object("fmd1")

In [36]:
for l in range(fmd2.L):
    print((0.23+0.1*l)*fmd2.mdld('origin', 'train', 1, l))
print()
for l in range(fmd2.L):
    print((0.23+0.1*l)*fmd2.normld('origin', 'train', 2, l))
print()
for l in range(fmd2.L):
    print((0.23+0.1*l)*fmd2.mmsld('origin', 'train', 3, l))

In [ ]:
import pickle
import gzip

with open("fmd1.p", "wb") as f:
    pickle.dump(fmd1, f, pickle.HIGHEST_PROTOCOL)

with gzip.open("fmd1_gzip.p", "wb") as f:
    pickle.dump(fmd1, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open("fmd1.p", "rb") as f:
    fmd2 = pickle.load(f)

In [ ]:
hi = open("hi.txt", 'w')
hi.write("dss1\n")
hi.write("dss2\n")
hi.write("dss3\n")
hi.write("dss4\n")
hi.write("dss5")
hi.close()

hi = open("hi.txt", "r")
hi_r = hi.read()

print(hi_r.split('\n'))
hi.close()

['dss1', 'dss2', 'dss3', 'dss4', 'dss5']
